In [1]:
import os

In [2]:
from dotenv import load_dotenv
load_dotenv()
HF_TOKEN=os.getenv("HF_TOKEN")

In [3]:
HUGGINGFACE_REPO_ID="mistralai/Mistral-7B-Instruct-v0.3"

# Setup LLM (Mistral with huggingface)

In [4]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEmbeddings

In [5]:
def load_llm(huggingface_repo_id):
    llm=HuggingFaceEndpoint(
        repo_id=huggingface_repo_id,
        temperature=0.5,
        model_kwargs={"token":HF_TOKEN,
                      "max_length":512}
    )
    return llm

# Connect LLM With FAISS and Create chain

Create custom prompt

In [6]:
CUSTOM_PROMPT_TEMPLATE = """
Use the pieces of information provided in the context to answer user's question.
If you dont know the answer, just say that you dont know, dont try to make up an answer.
Dont provide anything out of the given context

Context:{context}
Question:{question}

Start the answer directly. No small talk please.
"""

In [7]:
def set_custom_prompt(custom_prompt_template):
    promt=PromptTemplate(template=custom_prompt_template, input_variables=['context','question'])
    return promt

Load Database

In [8]:
from langchain_community.vectorstores import FAISS

In [9]:
DB_FAISS_PATH='vectorstore/db_faiss'

In [10]:
embedding_model=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

c:\Users\Deep\.conda\envs\vitabot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
db=FAISS.load_local(DB_FAISS_PATH, embedding_model, allow_dangerous_deserialization=True)

Create QA Chain

In [12]:
qa_chain=RetrievalQA.from_chain_type(
    llm=load_llm(HUGGINGFACE_REPO_ID),
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={'k':3}),
    return_source_documents=True,
    chain_type_kwargs={'prompt':set_custom_prompt(CUSTOM_PROMPT_TEMPLATE)}
)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Invoke with a single query

In [13]:
user_query=input("Write Query Here:")
response=qa_chain.invoke({'query':user_query})
print("RESULT:", response['result'])
print("Source Document:", response['source_documents'])

c:\Users\Deep\.conda\envs\vitabot\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


RESULT: 
Cancer can be cured through surgical removal of the tumor, which is usually the best chance for a surgical cure, particularly for the first operation. Chemotherapy may also be used in the treatment of certain cancers. Radiation may occasionally be used, particularly for tumors in the brain.
Source Document: [Document(id='71c15823-4a39-4146-92d7-08ec62d7f4db', metadata={'producer': 'GPL Ghostscript 9.10', 'creator': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'moddate': '2017-05-01T10:37:35-07:00', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'source': 'data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND (1).pdf', 'total_pages': 759, 'page': 26, 'page_label': '27'}, page_content='curative for some stomach, genital/urinary, thyroid,\nbreast, skin, and central nervous system cancers. The best\nchance for a surgical cure is usually with the first opera-\nGALE ENCYCLOPEDIA OF MEDICINE 2638\nCancer therapy, definitive'), Document(id='081e0d1a-36eb-4320-b411-0f01b3e9db6e